# LabelStudio annotations to TSV

> "raw output"

- branch: master
- hidden: true
- comments: false
- categories: [labelstudio, trim, csv]

Version of [this]({% post_url 2024-09-25-trim-labelstudio-annotations-write-tsv %}) that does not trim the outputs.

In [1]:
def slurpfile(filename) -> str:
    with open(filename) as inf:
        return inf.read().strip()

The API key is just read from `label_studio_mine` in the current directory (relative to the notebook). It's available under "Accounts & Settings" in the user menu, top right of the screen.

In [2]:
timecode_dir = "/Users/joregan/timecode_cut"
output_dir = "/tmp/textgrid_cut"
host = "http://130.237.3.107:8080/api/"
api_token: str = slurpfile("label_studio_mine")

In [3]:
from pathlib import Path

timecode_path = Path(timecode_dir)
output_path = Path(output_dir)

In [4]:
def get_timecode_offsets(filename):
    if type(filename) == Path:
        filename = str(filename)

    with open(filename) as inf:
        lines = [l.strip() for l in inf.readlines()]
        assert lines[0] == ",Frame,Time (Seconds),TimeCode", f"CSV file ({filename}) seems to be incorrect"
        p_start = lines[1].split(",")
        start = float(p_start[2])
        p_end = lines[-1].split(",")
        end = float(p_end[2])
        return start, end

In [5]:
import requests
import json
from pathlib import Path

headers = {
    "Authorization": f"Token {api_token}"
}

FIXME: need a better way to get these than hardcoding a list, but that will take a bunch of reading API docs

In [6]:
IDS = [
    311,
    308,
    312,
    313,
    309,
    310,
    314,
    315,
    317,
    316,
    318,
    319,
    320,
    322,
    321,
    324,
    326,
    327,
    330,
    329,
    331,
    332,
    333,
    334,
    335,
    338,
    340,
    328,
    337,
    325,
    341,
    323,
    336,
    164,
    165,
    166,
    167,
    168,
    169,
    170,
    171,
    172,
    173,
    174,
    175,
    176,
    177,
    178
]

In [7]:
def get_task(task_id):
    ep = f"{host}tasks/{task_id}"
    req = requests.get(ep, headers=headers)
    if req.status_code != 200:
        return {}
    data = json.loads(req.text)
    return data

In [8]:
def get_annotation(annot_it):
    ep = f"{host}annotations/{annot_it}"
    req = requests.get(ep, headers=headers)
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [37]:
data = get_annotation(264)

In [9]:
def combine_labels(data):
    combined = {}

    if "result" in data:
        for res in data["result"]:
            if not res["id"] in combined:
                combined[res["id"]] = res
            else:
                if "text" in res["value"]:
                    combined[res["id"]]["value"]["text"] = res["value"]["text"]
                elif "labels" in res["value"]:
                    combined[res["id"]]["value"]["labels"] = res["value"]["labels"]
    return combined

In [10]:
def adjust_times_write_tsv(data):
    task = data["task"]
    task_data = get_task(task)
    if "data" in task_data and "audio" in task_data["data"]:
        orig_file = task_data["data"]["audio"]
        parts = orig_file.split("/")
        orig_file = parts[-1]
    if orig_file:
        out_part = orig_file.replace(".wav", ".tsv")
    else:
        return []

    out_file = output_path / out_part
    if not output_path.is_dir():
        output_path.mkdir()

    results = []

    combined = combine_labels(data)

    for item in combined:
        val = combined[item]["value"]
        if not "labels" in val:
            continue
        if not "Speech" in val["labels"]:
            continue
        start = val["start"]
        end = val["end"]
        text = val["text"]
        if len(text) > 1:
            for t in text:
                if not (t.startswith("/") and t.endswith("/")):
                    text = t
        else:
            text = text[0]
        results.append((start, end, text))

        
    with open(out_file, "w") as outf:
        for res in results:
            outf.write("\t".join([str(x) for x in list(res)]) + "\n")
        

In [11]:
for transcription in IDS:
    data = get_annotation(transcription)
    if not "task" in data:
        print("Error with task", transcription)
        continue
    else:
        adjust_times_write_tsv(data)